In [1]:
%autosave 10
import time

Autosaving every 10 seconds


In [2]:
import openpyxl
import pandas as pd

# Please ensure that the Length of the Final Cuts are rounded off considerably

In case of mismatch due to algorithm rounding off after the Pattern Generation sheet is obtained, it may be manually matched

In [3]:
DashBoard_Obj=pd.read_excel("DashBoard.xlsx","Objective",index_col=0)
print(DashBoard_Obj)

                                                    Value  \
Parameter                                                   
Problem                                                20   
Objective                                               4   
Number of Combinations to be Generated                 40   
Timelimit for each of the smallest problems            30   
Timelimit for the Large CSP problems                  500   
Maximum Number of Unique Standard Length usages         2   
Maximum Number of Unique Pattern usages per Sta...     50   

                                                                                              Remarks  
Parameter                                                                                              
Problem                                                                  Diameter of the Bars (in mm)  
Objective                                           Case Type 4 = Generating the Top Minimum Waste...  
Number of Combinations to be Gener

In [4]:
# Create a dictionary from the index and a specific column
Decisions = DashBoard_Obj['Value'].to_dict()
print(Decisions)
# Decisions["Maximum Number of (Unique) Standard Length usages"]

{'Problem': 20, 'Objective': 4, 'Number of Combinations to be Generated': 40, 'Timelimit for each of the smallest problems': 30, 'Timelimit for the Large CSP problems': 500, 'Maximum Number of Unique Standard Length usages': 2, 'Maximum Number of Unique Pattern usages per Standard Length': 50}


In [5]:
DashBoard_Dataset_Cuts=pd.read_excel("DashBoard.xlsx","Dataset_Cuts",index_col=0)
print(DashBoard_Dataset_Cuts)

                         Total Requirements
Lengths of Cuts Allowed                    
1.810                                     8
2.730                                    59
2.830                                  2134
2.930                                  1443
2.950                                   954
2.980                                   301
3.030                                   332
3.130                                   612
3.892                                    48
3.893                                    24
3.911                                    36
3.914                                    48
7.090                                     4
7.326                                     8
7.328                                    16
7.337                                    32
7.350                                    16
7.353                                    16
7.390                                    16
7.391                                    10
7.626                           

In [6]:
DashBoard_Dataset_StandardLengths=pd.read_excel("DashBoard.xlsx","Dataset_StandardLengths",index_col=0)
#print(type(DashBoard_Dataset_StandardLengths))
print(DashBoard_Dataset_StandardLengths)

                                Associated Cost
Total Standard Lengths Allowed                 
7.0                                    0.219911
7.5                                    0.235619
11.5                                   0.361283
12.0                                   0.376991


In [7]:
length_of_cuts_allowed = DashBoard_Dataset_Cuts.index.tolist()
print(length_of_cuts_allowed)

[1.81, 2.73, 2.83, 2.93, 2.95, 2.98, 3.03, 3.13, 3.892, 3.893, 3.911, 3.914, 7.09, 7.326, 7.328, 7.337, 7.35, 7.353, 7.39, 7.391, 7.626, 7.676, 7.766, 12.0]


In [8]:
requirements = DashBoard_Dataset_Cuts["Total Requirements"]
print(requirements)
#requirements[1.2]

Lengths of Cuts Allowed
1.810        8
2.730       59
2.830     2134
2.930     1443
2.950      954
2.980      301
3.030      332
3.130      612
3.892       48
3.893       24
3.911       36
3.914       48
7.090        4
7.326        8
7.328       16
7.337       32
7.350       16
7.353       16
7.390       16
7.391       10
7.626        9
7.676       24
7.766       24
12.000      12
Name: Total Requirements, dtype: int64


In [9]:
total_max_standardlengths_allowed = DashBoard_Dataset_StandardLengths.index.tolist()
print(total_max_standardlengths_allowed)

[7.0, 7.5, 11.5, 12.0]


In [10]:
combinations = range(1,int(Decisions["Number of Combinations to be Generated"])+1)
print(combinations)

range(1, 41)


In [11]:
#max_time_in_seconds = Decisions["Timelimit for each of the smallest problems"]

#Timelimit_for_each_final_CSP_problem_minimizing_WCSP = Decisions["Timelimit for the Large CSP problems"]

# Generating Combinations Depending on Objectives

In [12]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
start_time_is_this=time.time()
print(start_time_is_this)

1702800713.676103


# Constraints dependant on Objectives

In [14]:
if Decisions["Objective"] == 4:
    M = 10000000
    X_Standards = {}
    
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        
        for c in combinations:
            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                if cc < c:
                    for l in length_of_cuts_allowed:
                        m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                        g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                        prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                        prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                        prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                        prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                        prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                        prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                    prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            waste_generated = prob.objVal

            for l in length_of_cuts_allowed:
                X_store[c, l] = x[l].x
                if X_store[c, l] > 0:
                    print("\t X_store[", c, ",", l, "] = ", X_store[c, l])

            print("\t \t Total Waste Generated: ", waste_generated)
            X_store[c, "Waste Objective"] = waste_generated
            print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem of Standard length: ", max_length)

        X_Standards[max_length] = X_store.copy()

Max Standard Length Allowed:  7.0
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-10
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 24 columns and 24 nonzeros
Model fingerprint: 0x171e6324
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 1e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+00]
Found heuristic solution: objective 1.5700000
Presolve removed 0 rows and 12 columns
Presolve time: 0.00s
Presolved: 1 rows, 12 columns, 12 nonzeros
Variable types: 0 continuous, 12 integer (4 binary)
Found heuristic solution: objective 0.2500000

Root relaxation: objective 0.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cu

Variable types: 96 continuous, 120 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 552 rows and 196 columns
Presolve time: 0.01s
Presolved: 29 rows, 20 columns, 104 nonzeros
Variable types: 0 continuous, 20 integer (12 binary)
Found heuristic solution: objective 0.3500000

Root relaxation: objective 0.000000e+00, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.35000    0.00000   100%     -    0s
H    0     0                       0.1060000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.10600    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.

  RHS range        [1e+00, 1e+07]
Presolve removed 1104 rows and 380 columns
Presolve time: 0.01s
Presolved: 57 rows, 28 columns, 196 nonzeros
Variable types: 0 continuous, 28 integer (20 binary)
Found heuristic solution: objective 0.3500000

Root relaxation: objective 0.000000e+00, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.35000    0.00000   100%     -

Thread count was 4 (of 4 available processors)

Solution count 4: 0.157 0.159 0.35 0.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.570000000000e-01, best bound 1.570000000000e-01, gap 0.0000%
	 X_store[ 12 , 2.95 ] =  1.0
	 X_store[ 12 , 3.893 ] =  1.0
	 	 Total Waste Generated:  0.15700000000000003
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  7.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1741 rows, 600 columns and 4344 nonzeros
Model fingerprint: 0xaac54331
Variable types: 288 continuous, 312 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 1656 rows and 564 columns
Presolve time: 0.0


Optimize a model with 2176 rows, 744 columns and 5424 nonzeros
Model fingerprint: 0xf17cd67b
Variable types: 360 continuous, 384 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 2070 rows and 702 columns
Presolve time: 0.02s
Presolved: 106 rows, 42 columns, 357 nonzeros
Variable types: 0 continuous, 42 integer (34 binary)
Found heuristic solution: objective 0.4300000
Found heuristic solution: objective 0.4000000

Root relaxation: objective -4.440892e-16, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0    1    0.40000   -0.00000   100%     -    0s
     0     0   -0.00000    0    5    0.40000   -0.00000   100%     -    0s
     0     0    0.00000    0    5    0.4000

...

Variable types: 0 continuous, 139 integer (69 binary)
Found heuristic solution: objective 0.6100000

Root relaxation: objective 0.000000e+00, 131 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2    0.61000    0.00000   100%     -    0s
H    0     0                       0.2200000    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.22000    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.22000    0.00000   100%     -    0s
     0     2    0.00000    0    2    0.22000    0.00000   100%     -    0s
H   36    69                       0.1200000    0.00000   100%   3.2    0s
H   44    69                       0.0800000    0.00000   100%   3.5    0s
*  216    69               8       0.0690000    0.00000   100%   5.6    0s
H  708    30                       0.0600000    0.00000   1

*  609    85              24       0.0600000    0.00000   100%   4.3    0s

Cutting planes:
  Learned: 83
  Gomory: 1
  Cover: 6
  Implied bound: 26
  RLT: 1

Explored 2170 nodes (9425 simplex iterations) in 0.94 seconds (0.26 work units)
Thread count was 4 (of 4 available processors)

Solution count 7: 0.06 0.069 0.07 ... 0.61

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e-02, best bound 6.000000000000e-02, gap 0.0000%
	 X_store[ 27 , 2.93 ] =  1.0
	 X_store[ 27 , 2.95 ] =  1.0
	 X_store[ 27 , 3.03 ] =  2.0
	 	 Total Waste Generated:  0.05999999999999872
	 	 Total Time Taken:  1.0  seconds, for this smallest problem of Standard length:  12.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3916 rows, 1320 columns and 9744 nonzeros
Model fingerprint: 0xd8a9914a
Varia

Found heuristic solution: objective 0.8100000
Found heuristic solution: objective 0.6900000

Root relaxation: objective 0.000000e+00, 186 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    3    0.69000    0.00000   100%     -    0s
H    0     0                       0.3420000    0.00000   100%     -    0s
H    0     0                       0.2860000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.28600    0.00000   100%     -    0s
     0     0    0.00000    0   10    0.28600    0.00000   100%     -    0s
H    0     0                       0.1700000    0.00000   100%     -    0s
     0     0    0.00000    0   10    0.17000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.17000    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.17000    0.00000   100%     


Cutting planes:
  Learned: 56
  Cover: 2
  Implied bound: 16
  Mod-K: 7
  Relax-and-lift: 4

Explored 1344 nodes (11033 simplex iterations) in 0.90 seconds (0.25 work units)
Thread count was 4 (of 4 available processors)

Solution count 6: 0.08 0.119 0.12 ... 1.3

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e-02, best bound 8.000000000000e-02, gap 0.0000%
	 X_store[ 34 , 2.83 ] =  1.0
	 X_store[ 34 , 2.93 ] =  1.0
	 X_store[ 34 , 3.03 ] =  1.0
	 X_store[ 34 , 3.13 ] =  1.0
	 	 Total Waste Generated:  0.08000000000000185
	 	 Total Time Taken:  0.9  seconds, for this smallest problem of Standard length:  12.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4931 rows, 1656 columns and 12264 nonzeros
Model fingerprint: 0xbbe79fdb
Variable types: 816 continuous, 840 int

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5366 rows, 1800 columns and 13344 nonzeros
Model fingerprint: 0x3b5ca307
Variable types: 888 continuous, 912 integer (888 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 4783 rows and 1595 columns
Presolve time: 0.11s
Presolved: 583 rows, 205 columns, 2353 nonzeros
Variable types: 0 continuous, 205 integer (102 binary)
Found heuristic solution: objective 1.3000000
Found heuristic solution: objective 0.8000000
Found heuristic solution: objective 0.2200000

Root relaxation: objective 0.000000e+00, 203 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    5    0.22000    0.00000   100%  

In [15]:
max_combination_for_each_standard = {}

if Decisions["Objective"] == 4:
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        combination_previous_end_number = Decisions["Number of Combinations to be Generated"]
        current_combination_number = combination_previous_end_number

        for ll in length_of_cuts_allowed:
            if ll > max_length:
                continue

            current_combination_number += 1

            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 0
            prob.addConstr(x[ll] >= 1)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_Standards[max_length][cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_Standards[max_length][cc, l])

                    prob.addConstr(x[l] - X_Standards[max_length][cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_Standards[max_length][cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_Standards[max_length][cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_Standards[max_length][cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            for cc in range(combination_previous_end_number + 1, current_combination_number):
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                    prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            
            
            
            if prob.SolCount:
                waste_generated = prob.objVal

                for l in length_of_cuts_allowed:
                    X_store[current_combination_number, l] = x[l].x
                    if X_store[current_combination_number, l] > 0:
                        print("\t X_store[", current_combination_number, ",", l, "] = ", X_store[current_combination_number, l])

                print("\t \t Total Waste Generated: ", waste_generated,
                      " during mandatory unique usage of cut-length: ", ll, "in standard length: ", max_length)
                X_store[current_combination_number, "Waste Objective"] = waste_generated
                print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem")
            else:
                current_combination_number-=1

        max_combination_for_each_standard[max_length] = current_combination_number
        X_Standards[max_length].update(X_store)

Max Standard Length Allowed:  7.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5802 rows, 1944 columns and 14425 nonzeros
Model fingerprint: 0xe107d084
Variable types: 960 continuous, 984 integer (960 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 5759 rows and 1924 columns
Presolve time: 0.03s
Presolved: 43 rows, 20 columns, 104 nonzeros
Variable types: 0 continuous, 20 integer (19 binary)
Found heuristic solution: objective 1.5700000
Found heuristic solution: objective 1.2760000

Root relaxation: objective 1.161243e-01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 6672 rows and 2232 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.02 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1.24 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.240000000000e+00, best bound 1.240000000000e+00, gap 0.0000%
	 X_store[ 47 , 2.73 ] =  1.0
	 X_store[ 47 , 3.03 ] =  1.0
	 	 Total Waste Generated:  1.2400000000000002  during mandatory unique usage of cut-length:  3.03 in standard length:  7.0
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

     0     0    0.06667    0    7    0.90000    0.06667  92.6%     -    0s
     0     0    0.06667    0    7    0.90000    0.06667  92.6%     -    0s
     0     0    0.06667    0   11    0.90000    0.06667  92.6%     -    0s
     0     0    0.06667    0   11    0.90000    0.06667  92.6%     -    0s
     0     0    0.06667    0   11    0.90000    0.06667  92.6%     -    0s
     0     0    0.06667    0   11    0.90000    0.06667  92.6%     -    0s
     0     2    0.11933    0   11    0.90000    0.11933  86.7%     -    0s

Cutting planes:
  Learned: 1
  Cover: 3
  MIR: 3
  RLT: 5

Explored 9 nodes (141 simplex iterations) in 0.18 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.9 1.05 1.15 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e-01, best bound 9.000000000000e-01, gap 0.0000%
	 X_store[ 41 , 1.81 ] =  2.0
	 X_store[ 41 , 2.98 ] =  1.0
	 	 Total Waste Generated:  0.9000000000000004  during mandatory unique usa

	 X_store[ 46 , 2.98 ] =  1.0
	 X_store[ 46 , 3.13 ] =  1.0
	 	 Total Waste Generated:  1.3900000000000006  during mandatory unique usage of cut-length:  2.98 in standard length:  7.5
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6672 rows, 2232 columns and 16585 nonzeros
Model fingerprint: 0xc12cadf3
Variable types: 1104 continuous, 1128 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 6672 rows and 2232 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.01 work units)
Thread count was 1 (of 4 available pro

Model fingerprint: 0xcb3cbfdd
Variable types: 1248 continuous, 1272 integer (1248 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7493 rows and 2520 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7542 rows, 2520 columns and 18745 nonzeros
Model fingerprint: 0xf9a3f7f1
Variable types: 1248 continuous, 1272 integer (1248 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bou

...

  Cover: 14
  MIR: 4
  StrongCG: 2
  RLT: 1

Explored 149 nodes (944 simplex iterations) in 0.32 seconds (0.07 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.09 0.26 1.48 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e-02, best bound 9.000000000000e-02, gap 0.0000%
	 X_store[ 45 , 2.95 ] =  2.0
	 X_store[ 45 , 2.98 ] =  1.0
	 X_store[ 45 , 3.03 ] =  1.0
	 	 Total Waste Generated:  0.08999999999999986  during mandatory unique usage of cut-length:  2.95 in standard length:  12.0
	 	 Total Time Taken:  0.3  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6527 rows, 2184 columns and 16225 nonzeros
Model fingerprint: 0x9f0cdb58
Variable types: 1080 continuous, 1104 integer (1080 binary)
Coefficient stati

Thread count was 4 (of 4 available processors)

Solution count 6: 0.119 0.137 0.169 ... 0.95

Optimal solution found (tolerance 1.00e-04)
Best objective 1.190000000000e-01, best bound 1.190000000000e-01, gap 0.0000%
	 X_store[ 48 , 1.81 ] =  1.0
	 X_store[ 48 , 3.03 ] =  1.0
	 X_store[ 48 , 3.13 ] =  1.0
	 X_store[ 48 , 3.911 ] =  1.0
	 	 Total Waste Generated:  0.11899999999999977  during mandatory unique usage of cut-length:  3.13 in standard length:  12.0
	 	 Total Time Taken:  0.5  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6962 rows, 2328 columns and 17305 nonzeros
Model fingerprint: 0x06828502
Variable types: 1152 continuous, 1176 integer (1152 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Boun

Thread count was 4 (of 4 available processors)

Solution count 6: 0.169 0.199 0.219 ... 1.639

Optimal solution found (tolerance 1.00e-04)
Best objective 1.690000000000e-01, best bound 1.690000000000e-01, gap 0.0000%
	 X_store[ 51 , 1.81 ] =  1.0
	 X_store[ 51 , 2.98 ] =  1.0
	 X_store[ 51 , 3.13 ] =  1.0
	 X_store[ 51 , 3.911 ] =  1.0
	 	 Total Waste Generated:  0.16900000000000048  during mandatory unique usage of cut-length:  3.911 in standard length:  12.0
	 	 Total Time Taken:  0.2  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7397 rows, 2472 columns and 18385 nonzeros
Model fingerprint: 0xca0679dd
Variable types: 1224 continuous, 1248 integer (1224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bo


Optimize a model with 7977 rows, 2664 columns and 19825 nonzeros
Model fingerprint: 0x3b7575d3
Variable types: 1320 continuous, 1344 integer (1320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7963 rows and 2657 columns
Presolve time: 0.03s
Presolved: 14 rows, 7 columns, 33 nonzeros
Variable types: 0 continuous, 7 integer (6 binary)
Found heuristic solution: objective 0.1230000

Root relaxation: objective 4.496635e-02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0         0.12300    0.12300  0.00%     -    0s

Explored 1 nodes (2 simplex iterations) in 0.07 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 0.123 

Optimal solution

Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 8696 rows and 2900 columns
Presolve time: 0.03s
Presolved: 6 rows, 4 columns, 16 nonzeros
Variable types: 0 continuous, 4 integer (3 binary)
Found heuristic solution: objective 0.7540000

Root relaxation: objective 4.600000e-01, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.4600000    0.46000  0.00%     -    0s

Explored 1 nodes (0 simplex iterations) in 0.08 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 0.46 0.754 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.600000000000e-01, best bound 4.600000000000e-01, gap 0.0000%
	 X_store[ 61 , 3.914 ] =  1.0
	 X_store[ 61 ,

In [16]:
import winsound
winsound.Beep(500,1000)
max_combination_for_each_standard

{7.0: 52, 7.5: 52, 11.5: 63, 12.0: 63}

In [17]:
wb=openpyxl.Workbook()

sheet_for_Combinations = wb.active

row_number=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Length of Cut"

column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value="Combination "+str(column_number-1)
        column_number+=1




for l in length_of_cuts_allowed:
    row_number+=1
    sheet_for_Combinations.cell(row=row_number,column=1).value= l
    column_number=2
    for i,max_length in enumerate(total_max_standardlengths_allowed):
        #for c in range(1,current_combination_number+1):
        for c in range(1,max_combination_for_each_standard[max_length]+1):
            sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,l]
            column_number+=1
        
        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Standard Max Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= max_length
        column_number+=1
        

        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Wasted Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,"Waste Objective"]
        column_number+=1

In [18]:
#file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+str(max_time_in_seconds)+".xlsx"
file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+".xlsx"
wb.save(file_path)
print(file_path)

./Problem_20__CombinationGeneration_40__TimeLimit_.xlsx


In [19]:
X_Standards

{7.0: {(1, 1.81): -0.0,
  (1, 2.73): -0.0,
  (1, 2.83): -0.0,
  (1, 2.93): -0.0,
  (1, 2.95): -0.0,
  (1, 2.98): -0.0,
  (1, 3.03): 1.0,
  (1, 3.13): -0.0,
  (1, 3.892): -0.0,
  (1, 3.893): -0.0,
  (1, 3.911): 0.0,
  (1, 3.914): 1.0,
  (1, 7.09): 0.0,
  (1, 7.326): 0.0,
  (1, 7.328): 0.0,
  (1, 7.337): 0.0,
  (1, 7.35): 0.0,
  (1, 7.353): 0.0,
  (1, 7.39): 0.0,
  (1, 7.391): 0.0,
  (1, 7.626): 0.0,
  (1, 7.676): 0.0,
  (1, 7.766): 0.0,
  (1, 12.0): 0.0,
  (1, 'Waste Objective'): 0.05600000000000005,
  (2, 1.81): 0.0,
  (2, 2.73): 0.0,
  (2, 2.83): 0.0,
  (2, 2.93): 0.0,
  (2, 2.95): 0.0,
  (2, 2.98): 0.0,
  (2, 3.03): 1.0,
  (2, 3.13): 0.0,
  (2, 3.892): 0.0,
  (2, 3.893): 0.0,
  (2, 3.911): 1.0,
  (2, 3.914): 0.0,
  (2, 7.09): 0.0,
  (2, 7.326): 0.0,
  (2, 7.328): 0.0,
  (2, 7.337): 0.0,
  (2, 7.35): 0.0,
  (2, 7.353): 0.0,
  (2, 7.39): 0.0,
  (2, 7.391): 0.0,
  (2, 7.626): 0.0,
  (2, 7.676): 0.0,
  (2, 7.766): 0.0,
  (2, 12.0): 0.0,
  (2, 'Waste Objective'): 0.05900000000000016,
  (3

In [20]:
for max_length in total_max_standardlengths_allowed:
    #print(max_length)
    print(X_Standards[max_length])

{(1, 1.81): -0.0, (1, 2.73): -0.0, (1, 2.83): -0.0, (1, 2.93): -0.0, (1, 2.95): -0.0, (1, 2.98): -0.0, (1, 3.03): 1.0, (1, 3.13): -0.0, (1, 3.892): -0.0, (1, 3.893): -0.0, (1, 3.911): 0.0, (1, 3.914): 1.0, (1, 7.09): 0.0, (1, 7.326): 0.0, (1, 7.328): 0.0, (1, 7.337): 0.0, (1, 7.35): 0.0, (1, 7.353): 0.0, (1, 7.39): 0.0, (1, 7.391): 0.0, (1, 7.626): 0.0, (1, 7.676): 0.0, (1, 7.766): 0.0, (1, 12.0): 0.0, (1, 'Waste Objective'): 0.05600000000000005, (2, 1.81): 0.0, (2, 2.73): 0.0, (2, 2.83): 0.0, (2, 2.93): 0.0, (2, 2.95): 0.0, (2, 2.98): 0.0, (2, 3.03): 1.0, (2, 3.13): 0.0, (2, 3.892): 0.0, (2, 3.893): 0.0, (2, 3.911): 1.0, (2, 3.914): 0.0, (2, 7.09): 0.0, (2, 7.326): 0.0, (2, 7.328): 0.0, (2, 7.337): 0.0, (2, 7.35): 0.0, (2, 7.353): 0.0, (2, 7.39): 0.0, (2, 7.391): 0.0, (2, 7.626): 0.0, (2, 7.676): 0.0, (2, 7.766): 0.0, (2, 12.0): 0.0, (2, 'Waste Objective'): 0.05900000000000016, (3, 1.81): -0.0, (3, 2.73): -0.0, (3, 2.83): -0.0, (3, 2.93): -0.0, (3, 2.95): -0.0, (3, 2.98): -0.0, (3, 3.

# Main CSProblem starts here:~

(designed to also run separately from the above program, be cautious about the intermediate file name changes if done manually)

In [21]:
import gurobipy as gp
from gurobipy import GRB

from math import ceil

In [22]:
patterns_df = pd.read_excel(file_path, index_col=0)
#print(patterns_df)

In [23]:
# Get the last two rows into a new DataFrame
last_two_rows = patterns_df.tail(2).copy()  # tail(2) gets the last 2 rows

# Remove the last two rows from the original DataFrame
new_df = patterns_df.drop(patterns_df.tail(2).index)

In [24]:
# Find unique elements in the first row (index 0)
unique_elements_row = last_two_rows.iloc[0].unique()

print("Unique elements in the first row of last_two_rows:")
print(unique_elements_row)

Unique elements in the first row of last_two_rows:
[ 7.   7.5 11.5 12. ]


In [25]:
all_length_of_cuts_allowed = new_df.index.tolist() # we could have used length_of_cuts_allowed, but here we take data from the Excel Sheet as we may want to change something for Later (in the Excel Sheet manually) which is however not recommended nor necessary
all_length_of_cuts_allowed

[1.81,
 2.73,
 2.83,
 2.93,
 2.95,
 2.98,
 3.03,
 3.13,
 3.892,
 3.893,
 3.911,
 3.914,
 7.09,
 7.326,
 7.328,
 7.337,
 7.35,
 7.353,
 7.39,
 7.391,
 7.626,
 7.676,
 7.766,
 12]

# Waste, Cost, Standard, Pattern: The WCSP sequence of Minimization (4 sequential problems

In [26]:
# Constraints
sl_key__value_c_mapping_dict={}
set_of_encountered_sl=set()
for c in patterns_df:
    sl = last_two_rows[c]["Standard Max Length"]
    if sl in set_of_encountered_sl:
        sl_key__value_c_mapping_dict[sl].add(c)
    else:
        sl_key__value_c_mapping_dict[sl]={c}
        set_of_encountered_sl.add(sl)
    
#print(sl_key__value_c_mapping_dict)

# WASTE MINIMIZATION

In [27]:
# Create a Gurobi model
CSProb = gp.Model("Stock Cutting with Waste Minimization")

# Variables
x = {}

for c in patterns_df:
    x[c] = CSProb.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{c}]')


# Objective: Minimize Waste
CSProb.setObjective(gp.quicksum(x[c] * last_two_rows[c]["Wasted Length"] for c in patterns_df), sense=GRB.MINIMIZE)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb.addConstr(gp.quicksum(x[c] * new_df[c][l] for c in patterns_df) >= requirements[l])





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb.Params.MIPFocus=2


# Optimize the model
CSProb.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 24 rows, 230 columns and 568 nonzeros
Model fingerprint: 0xf1357940
Variable types: 0 continuous, 230 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [7e-03, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+03]
Found heuristic solution: objective 471.2600000
Presolve 

# COST MINIMIZATION

In [28]:
# Create a new Gurobi model
CSProb2 = gp.Model("Stock Cutting with Cost Minimization")

# Variables
x_2 = {}

for c in patterns_df:
    x_2[c] = CSProb2.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_2[{c}]')



# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb2.addConstr(gp.quicksum(x_2[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000, name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb2.addConstr(gp.quicksum(x_2[c] * new_df[c][l] for c in patterns_df) >= requirements[l])


# Objective 2
CSProb2.setObjective(gp.quicksum(x_2[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][last_two_rows[c]["Standard Max Length"]] for c in patterns_df),
                     sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb2.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb2.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb2.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb2.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb2.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb2.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb2.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb2.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb2.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb2.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb2.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb2.Params.MIPFocus=2


# Optimize the model
CSProb2.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 25 rows, 230 columns and 796 nonzeros
Model fingerprint: 0xa0e25936
Variable types: 0 continuous, 230 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 5e+00]
  Objective range  [2e-01, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+03]
Presolve removed 1 rows and 40 columns
Presolve time: 0.0

# STANDARDs MINIMIZATION

In [29]:
# STANDARDs MINIMIZATION
M=100000

# Create a new Gurobi model
CSProb3 = gp.Model("Stock Cutting with Unique Standard length usage minimization")

#Debugging Mode
CSProb3.setParam('OutputFlag', 1)

# Variables
x_3 = {}
b_3 = {}

for c in patterns_df:
    x_3[c] = CSProb3.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_3[{c}]')

for sl in unique_elements_row:
    b_3[float(sl)] = CSProb3.addVar(vtype=GRB.BINARY, name=f'b_3[{sl}]')

    
    
    
# Constraints

for sl in unique_elements_row:
    CSProb3.addConstr( gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_3[float(sl)])
    CSProb3.addConstr( M*b_3[float(sl)] >= gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    

    

# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000,name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb3.addConstr(gp.quicksum(x_3[c] * new_df[c][l] for c in patterns_df) >= requirements[l],name=f"Requirement const. for {l}")


# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000, name="Obj2 as Const.")


# New Objective 3 to Minimize the number of Unique Standard Length Usage
CSProb3.setObjective(gp.quicksum(b_3[float(sl)] for sl in unique_elements_row),sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb3.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb3.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb3.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb3.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb3.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb3.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb3.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb3.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb3.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb3.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb3.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb3.Params.MIPFocus=2

#CSProb3.Params.NumericFocus = 3  # Experiment with different values (0, 1, 2, 3)


# Optimize the model
CSProb3.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 34 rows, 234 columns and 1494 nonzeros
Model fingerprint: 0x93ec046b
Variable types: 0 continuous, 234 integer (4 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+03]
Presolve added 3 rows and 1 columns
Presolve time: 0.01s

In [30]:
# IMPORTANT

# #Identify Infeasible Constraints:
# #Use the infeasibility information (model.IIS) to identify the constraints causing infeasibility. This can be done as follows:

# CSProb3.computeIIS()
# print('IIS constraints:')
# for constr in CSProb3.getConstrs():
#     if constr.IISConstr:
#         print(constr.ConstrName)

In [31]:
# Print variable values
for var in CSProb3.getVars():
    print(f'{var.varName}: {var.x}')

# # Print constraint values
# for constr in CSProb3.getConstrs():
#     print(f'{constr.ConstrName}: {constr.Pi} (dual value)')

# Print objective value
print(f'Objective Value: {CSProb3.objVal}')

x_3[Combination 1]: -0.0
x_3[Combination 2]: -0.0
x_3[Combination 3]: -0.0
x_3[Combination 4]: -0.0
x_3[Combination 5]: -0.0
x_3[Combination 6]: -0.0
x_3[Combination 7]: -0.0
x_3[Combination 8]: -0.0
x_3[Combination 9]: -0.0
x_3[Combination 10]: -0.0
x_3[Combination 11]: -0.0
x_3[Combination 12]: -0.0
x_3[Combination 13]: -0.0
x_3[Combination 14]: -0.0
x_3[Combination 15]: -0.0
x_3[Combination 16]: -0.0
x_3[Combination 17]: -0.0
x_3[Combination 18]: -0.0
x_3[Combination 19]: -0.0
x_3[Combination 20]: -0.0
x_3[Combination 21]: -0.0
x_3[Combination 22]: -0.0
x_3[Combination 23]: -0.0
x_3[Combination 24]: -0.0
x_3[Combination 25]: -0.0
x_3[Combination 26]: -0.0
x_3[Combination 27]: -0.0
x_3[Combination 28]: -0.0
x_3[Combination 29]: -0.0
x_3[Combination 30]: -0.0
x_3[Combination 31]: -0.0
x_3[Combination 32]: -0.0
x_3[Combination 33]: -0.0
x_3[Combination 34]: -0.0
x_3[Combination 35]: -0.0
x_3[Combination 36]: -0.0
x_3[Combination 37]: -0.0
x_3[Combination 38]: -0.0
x_3[Combination 39]: 

# PATTERNs MINIMIZATION

In [32]:
# Create a new Gurobi model
CSProb4 = gp.Model("Stock Cutting with Minimum Unique Pattern usage")

# Variables
x_4 = {}
y_4 = {}
b_4 = {}

for c in patterns_df:
    x_4[c] = CSProb4.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_4[{c}]')
    y_4[c] = CSProb4.addVar(vtype=GRB.BINARY, name=f'y_4[{c}]')

for sl in unique_elements_row:
    b_4[sl] = CSProb4.addVar(vtype=GRB.BINARY, name=f'b_4[{sl}]')

# Constraints
for c in patterns_df:
    #CSProb4.addConstr(x_4[c] >= b_4[last_two_rows[c]["Standard Max Length"]]) #Wrong
    CSProb4.addConstr(x_4[c] >= y_4[c])
    #CSProb4.addConstr(M * b_4[last_two_rows[c]["Standard Max Length"]] >= x_4[c]) #Wrong
    CSProb4.addConstr(M * y_4[c] >= x_4[c])

    
# Constraints
for sl in unique_elements_row:
    CSProb4.addConstr( gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_4[float(sl)])
    CSProb4.addConstr( M*b_4[float(sl)] >= gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    
# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb4.addConstr(gp.quicksum(x_4[c] * new_df[c][l] for c in patterns_df) >= requirements[l])

# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000)




# Previous Objective 3: to Minimize the number of Unique Standard Length Usage
CSProb4.addConstr(gp.quicksum(b_4[sl] for sl in unique_elements_row) <= ceil(CSProb3.objVal *1000)/1000)


# New Objective 4 to Minimize the number of Unique Pattern Usage
CSProb4.setObjective(gp.quicksum(y_4[c] for c in patterns_df),sense=GRB.MINIMIZE)





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb4.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb4.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb4.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb4.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb4.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb4.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb4.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb4.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb4.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb4.Params.MIPGap = 0
print("Default MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb4.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb4.Params.MIPFocus=2


# Optimize the model
CSProb4.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Default MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 495 rows, 464 columns and 2418 nonzeros
Model fingerprint: 0xb33bbc95
Variable types: 0 continuous, 464 integer (234 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+03]
Presolve added 1 rows and 3 columns
Presolve time: 0.0

# Populate Result Excel Sheet

In [33]:
end_time_is_this=time.time()
print(end_time_is_this)

actual_time_taken = end_time_is_this - start_time_is_this
actual_time_taken

1702800870.6162388


156.94013571739197

In [34]:
winsound.Beep(500,1000)
wb_final=openpyxl.Workbook()

sheet_for_Usage = wb_final.active
sheet_for_Usage.title = 'Variable x and y'
sheet_for_b = wb_final.create_sheet(title='Variable b')
sheet_for_Objectives = wb_final.create_sheet(title='Objectives')

#sheet_for_x
row_number=1
sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage in Objective 1 (x)"
sheet_for_Usage.cell(row=row_number,column=4).value= "No. of Usage in Objective 2 (x_2)"
sheet_for_Usage.cell(row=row_number,column=5).value= "No. of Usage in Objective 3 (x_3)"
sheet_for_Usage.cell(row=row_number,column=6).value= "No. of Usage in Objective 4 (x_4)"
sheet_for_Usage.cell(row=row_number,column=7).value= "Unique Pattern Usages in Objective 4 (y_4)"

for c in patterns_df:
    row_number+=1
    sheet_for_Usage.cell(row=row_number,column=1).value=c
    sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
    sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
    sheet_for_Usage.cell(row=row_number,column=4).value=x_2[c].x
    sheet_for_Usage.cell(row=row_number,column=5).value=x_3[c].x
    sheet_for_Usage.cell(row=row_number,column=6).value=x_4[c].x
    sheet_for_Usage.cell(row=row_number,column=7).value=y_4[c].x
    #if x_3[c].x:
    #    print(c, " used " , x_3[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])


    
#sheet_for_b
row_number=1
sheet_for_b.cell(row=row_number,column=1).value= "Standard Length"
sheet_for_b.cell(row=row_number,column=2).value= "Utilization in Objective 3 (b-3)"
sheet_for_b.cell(row=row_number,column=3).value= "Utilization in Objective 4 (b_4)"

for sl in unique_elements_row:
    row_number+=1
    sheet_for_b.cell(row=row_number,column=1).value=sl
    sheet_for_b.cell(row=row_number,column=2).value=b_3[sl].x
    sheet_for_b.cell(row=row_number,column=3).value=b_4[sl].x
    
    
#sheet_for_Objectives
row_number=1
sheet_for_Objectives.cell(row=row_number,column=1).value= "Objective 1"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Objective 2"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Objective 3"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Objective 4"
sheet_for_Objectives.cell(row=row_number,column=5).value= "Total Time taken for the Pattern Generations and all Objective Computations"
row_number=2
sheet_for_Objectives.cell(row=row_number,column=1).value= "Minimum Total WASTE"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Minimum Total COST"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Minimum Unique STANDARD LENGTH Usage"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Minimum Unique PATTERN Usage"
sheet_for_Objectives.cell(row=row_number,column=5).value= "(Seconds)"
row_number=3
sheet_for_Objectives.cell(row=row_number,column=1).value= CSProb.objVal
sheet_for_Objectives.cell(row=row_number,column=2).value= CSProb2.objVal
sheet_for_Objectives.cell(row=row_number,column=3).value= CSProb3.objVal
sheet_for_Objectives.cell(row=row_number,column=4).value= CSProb4.objVal
sheet_for_Objectives.cell(row=row_number,column=5).value= actual_time_taken


Final_output_file_path = "./Prob"+str(Decisions["Problem"])+"_PatternUsage_with_WCSP_Minimization"+".xlsx"
wb_final.save(Final_output_file_path)

In [35]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
#     if x[c].x:
#         print(c, " used " , x[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])

        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Trim Loss"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb.objVal
# print("Objective Value:", CSProb.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumTrimLoss"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)

In [36]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x_2[c].x
#     if x_2[c].x:
#         print(c, " used " , x_2[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])
        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Total Cost"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb2.objVal
# print("Objective Value:", CSProb2.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumCost"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)